In [4]:
import numpy as np
import tensorflow as tf
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
import sklearn.preprocessing
import sklearn.decomposition
from sklearn.linear_model import LogisticRegressionCV
from sklearn.datasets import fetch_mldata

In [2]:
DATA_HOME = "./data"
np.random.seed(42)

In [3]:
# Load MNIST Data
mnist = fetch_mldata('MNIST original', data_home=DATA_HOME)
data_x = sklearn.preprocessing.scale(mnist.data)
x_train, x_test, y_train, y_test = train_test_split(data_x, mnist.target, test_size=0.1, random_state=42)

/Users/dennybritz/projects/venvs/tensorflow/lib/python3.5/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype uint8 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [ ]:
class Autoencoder:
    def initialize(x, hidden_dims=[32]):
        self.layers = [x]
        
        # Build hidden layers
        for i, layer in enumerate(hidden_dims):
            layer_idx = i + 1
            prev_layer = self.layers[layer_idx-1]
            prev_layer_dim = prev_layer.get_shape().as_list()[1]
            layer_dim = hidden_dims[layer_idx]
            with tf.variable_scope("hidden-{}".format(i)):
                W = tf.Variable(
                        tf.truncated_normal([prev_layer_dim, layer_dim]),
                        name="W")
                b = tf.Variable(tf.zeros(layer_dim), name="b")
                output = tf.nn.relu(tf.nn.xw_plus_b(prev_layer, W, b))
                self.layers.append(output)
        
        # Build output layer 
        with tf.variable_scope("output"):
            output_dim = self.layers[0].get_shape().as_list()[1]
            prev_layer = self.layers[-1]
            prev_layer_dim = hidden_dims[-1]
            W = tf.Variable(
                tf.truncated_normal([prev_layer_dim, output_dim]),
                name="W")
            b = tf.Variable(tf.zeros(layer_dim), name="b")
            output = tf.nn.relu(tf.nn.xw_plus_b(prev_layer, W, b))
            self.layers.append(output)
        
        # Squared loss function
        self.loss = tf.reduce_sum(tf.square(self.layers[0] - self.layers[-1]))
